In [2]:
import numpy as np
from itertools import combinations
from itertools import product

from stats import Stats

# Creating an object of the class
character = Stats(0, 'character base Stats', (15, 20, 0, 150))

dmgs = np.arange(0,15,1)
crits = np.arange(0,160,10)
maxis = np.arange(0,140,10)
crit_dmgs = np.arange(0,50,5)


combs = list(product(dmgs, crits, maxis, crit_dmgs))
combs_len = len(combs)

#   Generating items
print(f'Generating {combs_len} combinations')

items = []
for index, group in enumerate(combs):
    items.append(Stats(index, f'random item {index}', group))
print('Done')

#   Generating items conbinations
print(f'Generating {combs_len} combinations')

results = []
for item in items:
    results.append(character.equip(item))
print('Done')

ordered_res = sorted(results)

print(Stats.headers())
print(ordered_res[0])
print(ordered_res[-1])

Generating 33600 combinations
Done
Generating 33600 combinations


AttributeError: 'Stats' object has no attribute 'equip'

In [1]:
%reset -f

import numpy as np
from itertools import combinations
from itertools import product

from stats import Character, Stats

# Creating an object of the class

base_stats = {  
                'physical_attack':  1500,
                'magical_attack':   1500,
                'physical_defense': 500,
                'magical_defense':  500,
                'hp':               450000,
                
                'damage_percent':   0,
                'critical':         40,
                'maximize':         5,
                'critical_damage':  0
              }

test_stats = {  
                'physical_attack':  500,
                'magical_attack':   500,
                'physical_defense': 500,
                'magical_defense':  500,
                'hp':               10000,
                
                'damage_percent':   1,
                'critical':         25,
                'maximize':         15,
                'critical_damage':  3
              }

top     = Stats('top', 'Amethystine Top', 'Chrimson', test_stats)
bottom  = Stats('bottom', 'Amethystine Bottom', 'Chrimson', test_stats)
gloves  = Stats('gloves', 'Amethystine Gloves', 'Chrimson', test_stats)
shoes   = Stats('shoes', 'Amethystine Shoes', 'Chrimson', test_stats)



jeff = Character('_Char', 'my name is jeff', '', base_stats)


jeff.equip(top, 'top')
jeff.equip(bottom, 'bottom')
jeff.equip(gloves, 'gloves')
jeff.equip(shoes, 'shoes')



unnormalized = jeff.calculate()
print(unnormalized)

normalized = unnormalized.apply_normalization()
print(normalized)

set = unnormalized.apply_sets()
print(set)

normalized.get_damage()


[id]      |name                      |set       |phys_atk  |mag_atk   |phys_def  |mag_def   |hp        |dmg_perc  |critical  |maximize  |crit_dmg  
top       |Amethystine Top           |Chrimson  |       500|       500|       500|       500|     10000|         1|        25|        15|         3
bottom    |Amethystine Bottom        |Chrimson  |       500|       500|       500|       500|     10000|         1|        25|        15|         3
gloves    |Amethystine Gloves        |Chrimson  |       500|       500|       500|       500|     10000|         1|        25|        15|         3
shoes     |Amethystine Shoes         |Chrimson  |       500|       500|       500|       500|     10000|         1|        25|        15|         3
_Char     |my name is jeff           |          |      3500|      3500|      2500|      2500|    490000|         4|       140|        65|        12



[id]      |name                      |set       |phys_atk  |mag_atk   |phys_def  |mag_def   |hp        |dmg_p

[2.12628, <stats.Character at 0x268903fa410>]